In [1]:
from functools import lru_cache
from os import environ
from dotenv import load_dotenv
import pandas as pd
import requests

load_dotenv("../")

API_KEY = environ["RIOT_API"]

In [2]:
regions = {
    "na": "na1",
    "br": "br1",
    "eun": "eun1",
    "euw": "euw1",
    "jp": "jp1",
    "kr": "kr",
    "la1": "la1",
    "la2": "la2",
    "oc": "oc1",
    "ph": "ph2",
    "ru": "ru",
    "sg": "sg2",
    "th": "th2",
    "tr": "tr1",
    "tw": "tw2",
    "vn": "vn2",
}

In [37]:
class Summoner:
    """
    Returns summoner level data.
    """

    def __init__(self, region, summoner_name):
        self.data = self.get_data(region, summoner_name)
        self.id = self.data.id.to_string(index=False)
        self.account_id = self.data.accountId.to_string(index=False)
        self.puuid = self.data.puuid.to_string(index=False)
        self.name = self.data.name.to_string(index=False)
        self.profile_icon_id = self.data.profileIconId.to_string(index=False)
        self.revision_date = self.data.revisionDate.to_string(index=False)
        self.summoner_level = self.data.summonerLevel.to_string(index=False)
        self.masteries = self.get_mastery_data(region, self.id)

    @lru_cache
    def get_data(self, region, summoner_name):
        """
        Get the summoner data.
        https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}
        """
        summoner_url = (
            f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}"
            + "?api_key="
            + API_KEY
        )
        req = requests.get(summoner_url, timeout=60)
        data = pd.json_normalize(req.json())

        return data

    @lru_cache
    def get_mastery_data(self, region, id):
        """
        Get the summoner's mastery data.
        https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{id}
        """
        mastery_url = (
            f"https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{id}"
            + "?api_key="
            + API_KEY
        )
        req = requests.get(mastery_url, timeout=60)
        data = pd.DataFrame.from_dict(req.json())

        return data

In [41]:
yogurtsauce = Summoner("na1", "yogurtsauce")

In [42]:
yogurtsauce.data

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q,jwWp9hL3BhaB8yVFHiTiKpYprW-dte-xemlv_M8cMXFD3Q,0_Z8M5TjaB96qj5ncZ3ewtq3H6CrLW6znsMYjxQ0Cgcrjr...,yogurtsauce,5028,1687717779703,522


In [36]:
class League:
    """
    Data dragon data.
    """

    def __init__(self):
        self.champions = self.get_champions()

    @lru_cache
    def get_champions(self):
        """
        Get this patch's champions.
        https://ddragon.leagueoflegends.com/cdn/13.12.1/data/en_US/champion.json
        """
        champions_url = (
            "https://ddragon.leagueoflegends.com/cdn/13.12.1/data/en_US/champion.json"
        )
        req = requests.get(champions_url, timeout=60).json()
        data = pd.DataFrame(req["data"]).T.reset_index()
        data = pd.concat(
            [data, pd.json_normalize(data.stats), pd.json_normalize(data["info"])],
            axis=1,
        )
        data[["primary_role", "secondary_role"]] = pd.DataFrame(
            data.tags.tolist(), index=data.index
        )
        data = data.drop(["index", "info", "image", "stats", "tags", "id"], axis=1)

        data = data.rename(
            {
                "key": "champion_id",
                "partype": "resource",
                # info
                "attack": "info_attack",
                "defense": "info_defense",
                "magic": "info_magic",
                "difficulty": "info_difficulty",
                # stats
                # hp
                "hp": "hp",
                "hpperlevel": "hp_level",
                "hpregen": "hp_regen",
                "hpregenperlevel": "hpregen_level",
                # mp
                "mp": "mp",
                "mpperlevel": "mp_level",
                "mpregen": "mp_regen",
                "mpregenperlevel": "mpregen_level",
                # ad
                "attackdamage": "attack",
                "attackdamageperlevel": "attack_level",
                # crit
                "crit": "crit",
                "critperlevel": "crit_level",
                # armour
                "armor": "armor",
                "armorperlevel": "armor_level",
                # attack speed
                "attackspeed": "atk_spd",
                "attackspeedperlevel": "atk_spd_level",
                # ms
                "movespeed": "move_spd",
                # attack range
                "attackrange": "atk_range",
                # spellblock
                "spellblock": "spellblock",
                "spellblockperlevel": "spellblock_level",
            },
            axis=1,
        )
        data.insert(0, "champion_id", data.pop("champion_id"))

        return data


League().champions

,champion_id,version,name,title,blurb,resource,hp,hp_level,mp,mp_level,...,attack,attack_level,atk_spd_level,atk_spd,info_attack,info_defense,info_magic,info_difficulty,primary_role,secondary_role
0,266,13.12.1,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,650,114,0,0.0,...,60,5.00,2.500,0.651,8,4,3,4,Fighter,Tank
1,103,13.12.1,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,Mana,590,96,418,25.0,...,53,3.00,2.000,0.668,3,4,8,5,Mage,Assassin
2,84,13.12.1,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,Energy,570,119,200,0.0,...,62,3.30,3.200,0.625,5,3,8,7,Assassin,None
3,166,13.12.1,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",Mana,630,104,350,40.0,...,52,3.50,4.000,0.638,0,0,0,0,Marksman,Assassin
4,12,13.12.1,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,Mana,685,120,350,40.0,...,62,3.75,2.125,0.625,6,9,5,7,Tank,Support
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,221,13.12.1,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",Mana,630,110,250,45.0,...,53,1.30,2.000,0.658,8,5,3,6,Marksman,None
159,115,13.12.1,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",Mana,606,106,480,23.5,...,54,3.10,2.000,0.656,2,4,9,4,Mage,None
160,26,13.12.1,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...",Mana,574,96,452,50.0,...,52,3.00,2.130,0.625,2,5,8,6,Support,Mage
161,142,13.12.1,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...",Mana,630,106,425,25.0,...,58,3.30,2.500,0.625,1,7,8,5,Mage,Support
